# SVM 识别验证码

## 安装需要的包

In [1]:
!pip install --upgrade pip

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [2]:
pip install libsvm

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp37-cp37m-linux_x86_64.whl size=263568 sha256=39717c723c7f5df64ceb4de210f0a8c1eff57fb882d790388deeb7c84b182cc8
  Stored in directory: /root/.cache/pip/wheels/21/34/ba/f55106bd43fb569e3f95143d605d18a74ad82fb402d8e75edd
Successfully built libsvm
Note: you may need to restart the kernel to use updated packages.


## 数据处理

In [56]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-

import cv2
import numpy as np
import uuid
import os

def is_black_line(image,x,y):
    if min(image[y,x,0],image[y,x,1],image[y,x,2]) < 12:
        return True
    else:
        return False

def is_black_pix(image,x,y):
    b = image[y,x,0]
    g = image[y,x,1]
    r = image[y,x,2]

    avg = (int(b)+int(g)+int(r))/3
    if r <244 and abs(avg-b) <4 and abs(avg-g) <4 and abs(avg-4) <4 :
        return True
    else:
        return False

def is_backgroud(image,x,y):
    if image[y,x] > 160:
        return True
    else:
        return False

def find_next_line_pos(image,p):
    height,width = image.shape[:2]
    next_pos = []
    if p['x'] >= width -1 :
        return []

    if is_black_line(image,p['x']+1,p['y']):
        next_pos.append({'x':p['x']+1,'y':p['y']}) 
    if p['y'] > 0 and is_black_line(image,p['x']+1,p['y']-1):
        next_pos.append({'x':p['x']+1,'y':p['y']-1}) 
    if p['y'] < height - 1 and is_black_line(image,p['x']+1,p['y']+1):
        next_pos.append({'x':p['x']+1,'y':p['y']+1}) 
    
    return next_pos

def find_line_path(image,path,p):
    height,width = image.shape[:2]
    if p['x'] >= width -1:
        return True
    next_pos = find_next_line_pos(image,p)
    if len(next_pos) == 0 :
        return False
    for np in next_pos:
        result = find_line_path(image,path,np)
        if result is True:
            path.append(np)
            return True

    return False

def find_first_pos_of_line(image):
    pos = []
    height,width = image.shape[:2]
    for y in range(height):
        if is_black_line(image,0,y):
            pos.append({'x':0,'y':y})
   
    return pos

def get_pix_gray_from_rgb(image,p):
    b = image[p['y'],p['x'],0]
    g = image[p['y'],p['x'],1]
    r = image[p['y'],p['x'],2]
    gray = r*0.299 + g*0.587 + b*0.114

    return int(gray)

def clear_horizotal_line_noise(image):
    first_pos = find_first_pos_of_line(image)
    if len(first_pos) == 0:
        return
    height,width = image.shape[:2]
    for pos in first_pos:
        path = []
        find_line_path(image,path,pos)
        if len(path) > 0:
            for p in path:
                if p['y'] == 0:
                    image[p['y'],p['x']] = image[p['y']+1,p['x']]
                elif p['y'] == height -1:
                    image[p['y'],p['x']] = image[p['y']-1,p['x']]
                else:
                    gray_top = get_pix_gray_from_rgb(image,{'x':p['x'],'y':p['y']-1})
                    gray_bottom = get_pix_gray_from_rgb(image,{'x':p['x'],'y':p['y']+1})
                    if gray_top >= gray_bottom:
                        image[p['y'],p['x']] = image[p['y']-1,p['x']]
                    else:
                         image[p['y'],p['x']] = image[p['y']+1,p['x']]
            break

def clear_pix_noise(image):
    height,width = image.shape[:2]
    for y in range(height):
        for x in range(width):
            white_color_num = 0
            if image[y,x] <=20 :
                if x > 0 and is_backgroud(image,x-1,y):
                    white_color_num += 1
                if x < width - 1 and is_backgroud(image,x+1,y):
                    white_color_num += 1
                if y > 0 and is_backgroud(image,x,y-1):
                    white_color_num += 1
                if y < height -1 and is_backgroud(image,x,y+1):
                    white_color_num += 1
 
                if white_color_num >= 2:
                    image[y,x] = 255

def clear_pix_noise_by_median_blue(image,p):
    gray_color = []
    for k in range(-1,2):
        for j in range(-1,2):
            gray_color.append(image[p['y']+k,p['x']+j])

    sorted_gray_color=sorted(gray_color)
    image[p['y'],p['x']] = sorted_gray_color[4]

def clear_noise(image):
    clear_horizotal_line_noise(image)
    clear_color(image)

    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    clear_pix_noise(image)
    ret, image = cv2.threshold(image,127,255,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)

    return image

def clear_color(image):
    height,width = image.shape[:2]
    for y in range(height):
        for x in range(width):
            if is_black_pix(image,x,y):
                image[y,x,0] = 20
                image[y,x,1] = 20
                image[y,x,2] = 20


def get_row_rect1(image):
    height,width = image.shape[:2]
    h = [0,]*height
    for y in range(height):
        for x in range(width):
            s = image[y,x] 
            if s < 12:
                h[y] += 1
    in_line = False
    start_line = 0
    ####
    blank_distance = 1
    ####
    row_rect = (0,0)
    for i in range(len(h)):
        if not in_line and h[i]>=blank_distance:
            in_line = True
            start_line = i
        elif in_line and h[i]<blank_distance:
            row_rect = (start_line,i)
            break

    return row_rect

def get_row_rect(image):
    height,width = image.shape[:2]
    h = [0,]*height
 
    for y in range(height):
        for x in range(width):
            s = image[y,x] 
            if s < 12:
                h[y] += 1
    start_line = 0
    end_line = height -1

    for i in range(len(h)):
        if h[i] > 0:
            start_line = i
            break

    for i in range(len(h)):
        if h[height-1-i] > 0:
            end_line = height-1-i
            break

    return (start_line,end_line)

def get_col_rect(image):
    height,width = image.shape[:2]
    h = [0]*width
    for x in range(width):
        for y in range(1,height-1):
            s = image[y,x] 
            if s < 12:
                h[x] += 1
    col_rect = []
    in_line = False
    start_line = 0
    #####
    blank_distance = 1
    col_distance = 8
    ####
    for i in range(len(h)):
        if not in_line and h[i]>=blank_distance:
            in_line = True
            start_line = i
        elif in_line and h[i]<blank_distance and i-start_line > col_distance:
            rect = (start_line,i)
            col_rect.append(rect)
            in_line = False
            start_line = 0

    return col_rect

def get_block_image(image,col_rect):
    col_image = image[0:image.shape[0],col_rect[0]:col_rect[1]]
    #row_rect = (0,image.shape[0])
    row_rect = get_row_rect(col_image)
    #print row_rect
    if row_rect[1] != 0:
        block_image = image[row_rect[0]:row_rect[1],col_rect[0]:col_rect[1]]
    else:
        block_image = None
    return  block_image

def center_img(filename,height,width):
    img_org = cv2.imread(filename,0)
    img_new = np.zeros((height,width))
    img_new[...] = 255

    height_img_org,width_img_org = img_org.shape[:2]
    if height_img_org < height:
        y = (height - height_img_org) / 2
        yt = height_img_org
    else:
        y = 0
        yt = height
    if width_img_org < width:
        x = (width - width_img_org) / 2
        xt = width_img_org
    else:
        x = 0
        xt = width
    x = int(x)
    y = int(y)
    img_new[y:y+yt,x:x+xt] = img_org[0:yt,0:xt]
    cv2.imwrite(filename,img_new)

def split(filename,image_height,image_width):
    image =cv2.imread(filename)

    image = clear_noise(image)   
    col_rect = get_col_rect(image)
    #print col_rect
    for cols in col_rect:
        if cols[1] - cols[0] < 5:
            continue
        block_image = get_block_image(image,cols)
        if block_image is not None:
            new_image_filename = 'letters/'+str(uuid.uuid4())+'.png'
            cv2.imwrite(new_image_filename,block_image) 
            center_img(new_image_filename,image_height,image_width)  

def split_one_file(filename,image_height=13,image_width=9):
    image =cv2.imread(filename)

    image = clear_noise(image)   
    cv2.imwrite('/tmp/%s'%filename,image)   
    col_rect = get_col_rect(image)
    #print col_rect
    index = 1
    filenames = []
    for cols in col_rect:
        if cols[1] - cols[0] < 5:
            continue
        block_image = get_block_image(image,cols)
        if block_image is not None:
            new_image_filename = '/tmp/tmp_img_%s.png' %index
            cv2.imwrite(new_image_filename,block_image) 
            center_img(new_image_filename,image_height,image_width)   

            filenames.append(new_image_filename)
            index += 1

    return filenames

def main():
    image_height,image_width = 20,20
    #for i in range(10):
    #    split('test%s.png'%i)
    for file in os.listdir('./downloader/captchas'):
        f = './downloader/captchas/%s'%file
        if os.path.isfile(f) and f.endswith('.png'):
            split(f,image_height,image_width)
    
    print('done...')

if __name__ == '__main__':
    main()

done...


## SVM 训练

In [59]:
#!/usr/bin/env python
#-*- coding:utf-8 

import cv2
import os
import string
from libsvm.svmutil import *
from libsvm.svm import *

def get_feature(filename):
    image = cv2.imread(filename,0)
    height,width = image.shape[:2] # 13, 9
    
    result = []
    for y in range(0,height,1):
        for x in range(0,width,1):
            black_color = 0
            if image[y,x] < 127:
                black_color += 1
            result.append(black_color)

    return result

def get_train_features(labels):
    results = []
    for l in labels:
        image_path = './train/%s' %l
        features = []
        index = string.printable.find(l)
        for d in range(0,7):
            f = '%s%d.png'%(image_path,d)
            if os.path.isfile(f) and f.endswith('.png'):
                feature_data = get_feature(f)
                feature_data_lines = []
                #data ->: 1:X 2:Y 3:Z ...
                for j,data in enumerate(feature_data):
                    feature_data_lines.append('%s:%s'%(j+1,data))
                #data ->: index 1:X 2:Y 3:Z ...
                features.append('%s %s'%(index,' '.join(feature_data_lines)))
        results.extend(features)
        
    return results

def _model_train(svm_feature_filename,svm_model_filename):
    y, x = svm_read_problem(svm_feature_filename)
    m = svm_train(y,x,'-c 4')
    svm_save_model(svm_model_filename,m)
    
def _model_test(svm_model_filename,svm_test_filename):
    yt, xt = svm_read_problem(svm_test_filename)
    model = svm_load_model(svm_model_filename)
    p_label, p_acc, p_val = svm_predict(yt, xt, model)
    
    return p_label
    
def _model_test_one(svm_model_filename,feature_data):
    model = svm_load_model(svm_model_filename)
    y = [-1,]
    x = [feature_data]

    p_label, p_acc, p_val = svm_predict(y, x, model)
    return p_label

def model_test_for_file(svm_model_filename,filename):
    feature_data = get_feature(filename)
    label = _model_test_one(svm_model_filename,feature_data)

    code = int(label[0])
    
    # return code
    return string.printable[int(code)]


def main():
    svm_model_filename = 'svm_model.txt'
    #get train features:
    results = get_train_features('0123456789')

    import random
    random.shuffle(results)
    r_train = results[:int(len(results)*0.9)]
    r_test = results[int(len(results)*0.9):]
    #print len(r_train)
    #print len(r_test)
    with open('svm_train.txt','w') as f:
        for s in r_train:
            f.write(s)
            f.write(os.linesep)
    with open('svm_test.txt','w') as f:
        for s in r_test:
            f.write(s)
            f.write(os.linesep)

    _model_train('svm_train.txt',svm_model_filename)
    _model_test('svm_model.txt','svm_test.txt')
    

if __name__ == '__main__':
    main()


*
optimization finished, #iter = 8
nu = 0.153921
obj = -4.002142, rho = 0.195690
nSV = 5, nBSV = 1
*
optimization finished, #iter = 8
nu = 0.303987
obj = -7.903941, rho = 0.216762
nSV = 5, nBSV = 1
*
optimization finished, #iter = 10
nu = 0.329569
obj = -7.910525, rho = 0.480723
nSV = 6, nBSV = 1
*
optimization finished, #iter = 3
nu = 0.116341
obj = -3.024877, rho = 0.003553
nSV = 3, nBSV = 0
*
optimization finished, #iter = 5
nu = 0.147480
obj = -4.128158, rho = 0.024454
nSV = 4, nBSV = 0
*
optimization finished, #iter = 9
nu = 0.111651
obj = -3.125120, rho = 0.185436
nSV = 3, nBSV = 0
*
optimization finished, #iter = 9
nu = 0.258259
obj = -6.199101, rho = 0.381068
nSV = 4, nBSV = 0
*
optimization finished, #iter = 8
nu = 0.227643
obj = -6.374461, rho = 0.221181
nSV = 5, nBSV = 0
*
optimization finished, #iter = 1
nu = 0.099533
obj = -2.786916, rho = 0.000000
nSV = 2, nBSV = 0
*.*
optimization finished, #iter = 12
nu = 0.175413
obj = -4.211452, rho = -0.065857
nSV = 5, nBSV = 0
*
opt

## 测试

In [65]:
def main():
    svm_model_filename = 'svm_model.txt'
    #filename = 'test1.png'
    for file in os.listdir('./test/'):
        if file.endswith('.png'):
            filenames = split_one_file('./test/'+file)

            results = []
            for f in filenames:
                code = model_test_for_file(svm_model_filename,f)
                results.append(code)
            print('the filename is: %s , and the result is: ' %file, ''.join(results))


if __name__ == '__main__':
    main()

Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
the filename is: 9819.png , and the result is:  9819
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
the filename is: 9328.png , and the result is:  9328
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
the filename is: 1482.png , and the result is:  1482
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
the filename is: 9720.png , and the result is:  9720
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
the filename is: 4968.png , and the result is:  